# Basic summary of what this capstone will, and may accomplish

* Scrape data from indeed, and return job postings
* properly format location, title, requirements
* analyze what words and requirements occur the most
* Determine the quality of each job posting (how generic does it sound, and how common are the phrases that they are using) - this is important, but needs a bit of refinement

Stretch goals
* add input to allow users to specifiy the cities to look at

* add a map to visualize how many postings there are in each city 

Current Issues
* Sponsored job posts show up in data multiple times, and location data is missing
* Summary page is not formated properly, .split() doesn't fix it

# Indeed Job Scraper

So far, it can scrape a job's

* location
* title
* company name
* salary
* summary
* (html - data is unstructured, though)

would be nice, but idk how to implement

* responsibilities
* requirements

In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import time
import os

In [2]:
def parse(query,metro):
    
    # Target url, and n number of pages to scrape
    url = "https://www.indeed.com/jobs?q={}&l={}&start={}"
    n = 10
    
    # Create dataframe that we'll return later
    df = pd.DataFrame(columns=["Title","Location","Company","Salary", "Synopsis", "Query", "Metro"])
    
    # Scrape first n pages
    for i in range(0,n):
        html = requests.get(url.format(query, metro, i))
        time.sleep(1)
        soup = BeautifulSoup(html.content, 'html.parser', from_encoding="utf-8")
        
        for each in soup.find_all(class_= "result" ):
            try: 
                title = each.find(class_='jobtitle').text.replace('\n', '')
            except:
                title = 'None'
            try:
                location = each.find('span', {'class':"location" }).text.replace('\n', '')
            except:    
                try:
                    location = each.find('div', {'class':"location" }).text.replace('\n', '')
                except:
                    location = 'None'
            try: 
                company = each.find(class_='company').text.replace('\n', '')
            except:
                company = 'None'
            try:
                salary = each.find('span', {'class':'no-wrap'}).text
            except:
                salary = 'None'
            try:
                link = 'https://www.indeed.com/viewjob' + each.find('a', href=True)['href'][7:]
            except:    
                link = 'None'
            synopsis = each.find('span', {'class':'summary'}).text.replace('\n', '')
            df = df.append({'Title':title, 'Location':location, 'Company':company, 'Salary':salary, 'Synopsis':synopsis, 'Query': query, 'Metro': metro}, ignore_index=True)
    return df

In [3]:
# Lists of positions and locations to search
positions = ['data+analyst', 'data+scientist', 'data+engineer']
cities = ['washington+dc']

# Create the dataframe
data = pd.DataFrame()

# Scrape all positions in all cities specified, and append to dataframe
for position in positions:
    for city in cities:
        data = pd.concat([data, parse(position, city)])

In [4]:
# Save scraped data to a local directory
if os.path.exists("data/dc_indeed.csv"):
    os.remove("data/dc_indeed.csv")
data.to_csv('data/dc_indeed.csv', encoding='utf-8')

In [5]:
# Preview the data
data.head(10)

,Title,Location,Company,Salary,Synopsis,Query,Metro
0,Data Analyst,"Washington, DC",Chenega Corporation,None,Perform data entry...,data+analyst,washington+dc
1,Data Analyst,"Washington, DC",Callahan & Associates,None,Hands on experienc...,data+analyst,washington+dc
2,Product Modeling Analyst,"Chevy Chase, MD",GEICO,None,Statistical Modeli...,data+analyst,washington+dc
3,Data Scientist - Human Capital Analytics,"Bethesda, MD 20816",Leidos,None,"Review, assess dat...",data+analyst,washington+dc
4,Systems Analyst/Data Scientist,"Springfield, VA",Radiant Solutions,None,We build advanced ...,data+analyst,washington+dc
5,Business Intelligence Analyst,"Bethesda, MD",Common Securitization Solutions,None,Ability to determi...,data+analyst,washington+dc
6,Health Interventions and Claims Data Analyst,"Washington, DC 20007 (Georgetown area)",Innovation Center for Biomedical Infor...,None,We are working on applying data sc...,data+analyst,washington+dc
7,Data Analyst 1 - Petroleum Supply,"Washington, DC 20585",IMG Crown Energy Services Joint Venture,"\n $40,000 - $45,000 a year",Supporting the data validation tea...,data+analyst,washington+dc
8,Data Fellowship (Spring 2018),"Alexandria, VA",Echelon Insights,"\n $1,000 a month",Fellows with proven abilities in t...,data+analyst,washington+dc
9,Political Data Analyst - Outside Groups,"Washington, DC 20005 (Logan Circle area)",Data Trust,None,Data Trust is currently seeking a Data Analyst...,data+analyst,washington+dc


In [6]:
# Return size of dataframe
data.shape

(499, 7)